In [26]:
import os
import pandas as pd

path= r"E:\instacart_2017_05_01"
os.chdir(path)
os.listdir(path)
aisles=pd.read_csv("aisles.csv")
departments=pd.read_csv("departments.csv")
orders_product1 = pd.read_csv("order_products__part1.csv")
orders_product2 = pd.read_csv('order_products__part2.csv')
orders =pd.read_csv("orders.csv")
products =pd.read_csv("products.csv")

In [27]:
#we will append both orders excel
trans_df=orders_product2.append(orders_product1,ignore_index=True)
orders_product2.shape

(32434489, 4)

In [6]:
#number of customers?

len(orders.user_id.unique())

206209

In [ ]:
#percentage of customer placing more than  80 orders

In [10]:
list(orders)

['order_id',
 'user_id',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

In [11]:
customer=orders.groupby('user_id')['order_id'].count().reset_index()
customer.head()

,user_id,order_id
0,1,11
1,2,15
2,3,13
3,4,6
4,5,5


In [17]:
customer1=customer[customer['order_id']>80]
customer1.head()

,user_id,order_id
26,27,82
209,210,100
309,310,100
312,313,100
408,409,87


In [19]:
(len(customer1)/len(customer))*100

1.3719090825327702

In [34]:
#Which is more popular Organic Strawberries or Strawberries?

organic_straberries=trans_df[trans_df['product_id']==21137]
len(organic_straberries)

275577

In [43]:
len(organic_straberries)/len(trans_df.groupby('order_id').size())*100

8.235808854711614

In [44]:
Strawberries=trans_df[trans_df['product_id']==16797]

In [46]:
len(Strawberries)/len(trans_df.groupby('order_id').size())*100

4.466266975445618

In [63]:
len(Strawberries)

149445

In [49]:
#How likely is it to purshace banana if someone purchase Strawberries?

banana=trans_df[trans_df['product_id']==24852]
Strawberries=trans_df[trans_df['product_id']==16797]

In [55]:
Confidence=pd.merge(banana,Strawberries,left_on='order_id',right_on='order_id',how='inner')
len(Confidence)

43180

In [58]:
(len(Confidence)/len(Strawberries))*100

28.893572886346146

In [97]:
#How strong is the asscoiatoin between Organic Raspbarries and Organic Strawberries?

Organic_Ras=trans_df[trans_df['product_id']==27966]
Organic_Stra=trans_df[trans_df['product_id']==21137]

In [105]:
#Support of Organic Ras
A=len(Organic_Ras)/len(trans_df.groupby('order_id').size())
A

0.042617890829366754

In [106]:
#Support of Organic Stra
B=len(Organic_Stra)/len(trans_df.groupby('order_id').size())
B

0.08235808854711614

In [110]:
#Supoort of both

Suport=pd.merge(Organic_Ras,Organic_Stra,left_on='order_id',right_on='order_id',how='inner')
C=len(Suport)/len(trans_df.groupby('order_id').size())
C

0.01061928230710356

In [111]:
lift=C/(A*B)
lift

3.025498593297621

In [76]:
products[products['product_name']=='Organic Raspberry']

,product_id,product_name,aisle_id,department_id


In [ ]:
#Step=1 FIng the top 20 product

In [123]:
productcount=trans_df.groupby(['product_id']).size().reset_index()
productcount.head()
#Renaminig the 0 columns
productcount=productcount.rename(columns={0:"Freq"})
#Findin the top 20 prducts
productcount_top20=productcount.sort_values(by='Freq',ascending=False)
productcount_top20.head()

,product_id,Freq
24849,24852,491291
13173,13176,394930
21134,21137,275577
21900,21903,251705
47205,47209,220877


In [124]:
#Bringing the column of product name


newproductCountDf=pd.merge(productcount_top20,products,left_on='product_id',right_on='product_id',how='inner')
newDf=newproductCountDf[['product_id','product_name']]
newDf.head()

,product_id,product_name
0,24852,Banana
1,13176,Bag of Organic Bananas
2,21137,Organic Strawberries
3,21903,Organic Baby Spinach
4,47209,Organic Hass Avocado


In [126]:
#List for which i need to filter my trans_df


productID=newDf[['product_id']]
productID.head()

,product_id
0,24852
1,13176
2,21137
3,21903
4,47209


In [129]:
#Filtering the trans_df for these 20 product ID


df=trans_df[0:0]
for i in range (0,20):
    pId=productID.iloc[i]['product_id']
    stDf=trans_df[trans_df['product_id']==pId]
    df = df.append(stDf,ignore_index=False)
df.head()

,order_id,product_id,add_to_cart_order,reordered
77,10,24852,1,1
180,20,24852,6,0
190,22,24852,3,1
234,26,24852,2,1
414,52,24852,2,1


In [130]:
#Getting the names for each product ID


df=pd.merge(df,newDf,left_on='product_id',right_on='product_id',how='inner')
df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,10,24852,1,1,Banana
1,20,24852,6,0,Banana
2,22,24852,3,1,Banana
3,26,24852,2,1,Banana
4,52,24852,2,1,Banana


In [132]:
df.shape

(3466225, 5)

In [133]:
df['order_id'].nunique()

1801745

In [136]:
#Lets create the long table of order_id and product_name

basket=df.groupby(['order_id','product_name']).size().reset_index()
basket.head()

,order_id,product_name,0
0,1,Bag of Organic Bananas,1
1,1,Cucumber Kirby,1
2,1,Organic Hass Avocado,1
3,3,Organic Baby Spinach,1
4,5,Bag of Organic Bananas,1


In [137]:
#Converting the long data to wide data

basket1=basket.pivot('order_id','product_name',0)
basket1.head()

product_name,Apple Honeycrisp Organic,Bag of Organic Bananas,Banana,Cucumber Kirby,Large Lemon,Limes,Organic Avocado,Organic Baby Spinach,Organic Blueberries,Organic Fuji Apple,Organic Garlic,Organic Grape Tomatoes,Organic Hass Avocado,Organic Lemon,Organic Raspberries,Organic Strawberries,Organic Whole Milk,Organic Yellow Onion,Organic Zucchini,Strawberries
order_id,,,,,,,,,,,,,,,,,,,,
1,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [138]:
basket1.shape

(1801745, 20)

In [139]:
#FIll nan with 0

Final_basket=basket1.fillna(0)
Final_basket.head()

product_name,Apple Honeycrisp Organic,Bag of Organic Bananas,Banana,Cucumber Kirby,Large Lemon,Limes,Organic Avocado,Organic Baby Spinach,Organic Blueberries,Organic Fuji Apple,Organic Garlic,Organic Grape Tomatoes,Organic Hass Avocado,Organic Lemon,Organic Raspberries,Organic Strawberries,Organic Whole Milk,Organic Yellow Onion,Organic Zucchini,Strawberries
order_id,,,,,,,,,,,,,,,,,,,,
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [140]:
!pip install mlxtend

In [143]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [145]:
#Creating the frequenting occuring itemset


freq_itemset=apriori(Final_basket,min_support=0.01,use_colnames=True)

In [147]:
#Creating Rules

rules=association_rules(freq_itemset, metric='lift',min_threshold=1.05)

In [150]:
rules.sort_values(by='confidence',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Organic Fuji Apple),(Banana),0.051555,0.272675,0.019511,0.378441,1.387882,0.005453,1.170162
11,(Cucumber Kirby),(Banana),0.055351,0.272675,0.018227,0.329296,1.207649,0.003134,1.084420
13,(Organic Avocado),(Banana),0.102248,0.272675,0.030865,0.301866,1.107055,0.002985,1.041813
6,(Organic Raspberries),(Bag of Organic Bananas),0.079147,0.219193,0.023468,0.296508,1.352728,0.006119,1.109902
2,(Organic Hass Avocado),(Bag of Organic Bananas),0.122591,0.219193,0.035943,0.293199,1.337631,0.009072,1.104706
17,(Strawberries),(Banana),0.082945,0.272675,0.023966,0.288936,1.059634,0.001349,1.022868
0,(Apple Honeycrisp Organic),(Bag of Organic Bananas),0.048437,0.219193,0.013533,0.279391,1.274634,0.002916,1.083537
4,(Organic Lemon),(Bag of Organic Bananas),0.050646,0.219193,0.013015,0.256983,1.172407,0.001914,1.050861
54,(Organic Raspberries),(Organic Strawberries),0.079147,0.152950,0.019721,0.249174,1.629122,0.007616,1.128158
47,(Organic Lemon),(Organic Hass Avocado),0.050646,0.122591,0.012225,0.241389,1.969067,0.006017,1.156600


In [151]:
rules.shape

(58, 9)